# Contrastive Learning From Scratch - DistilBERT

An attempt to build contrastive learning model from scratch. Parts include:

- Loading and preparing Wiki-1M data for model input
- Contrastive learning model
  - Forward passing using pre-trained model
  - Constrastive layer
  - Calculate loss
- Training procedure
  - Default trainer optimizer
  - Default trainer hyper-parameters

In [1]:
import os

# Set Project home
PROJECT_HOME = os.path.join('/',
                            'notebooks',
                            'contrastive-learning-in-distilled-models')
%cd {PROJECT_HOME}

# Load project code
%reload_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, f'{PROJECT_HOME}/src')

import distilface

/notebooks/contrastive-learning-in-distilled-models


In [2]:
# Hyperparameters

pooling_method = 'avg_first_last'
batch_size = 64
epochs = 2
max_steps = 30_000
temperature = 0.05
learning_rate = 5e-5
max_len = 32
fp16 = True

In [3]:
# Parameters
pooling_method = "max_all_hidden"


## 1. Loading and Preparing Wiki-1M data

Use huggingface `datasets` library to load local file data.

In [4]:
import numpy as np

from datasets import load_dataset

data_files = {'train': 'data/training/wiki1m_for_simcse.txt'}
# data_files = {'train': 'data/training/wiki5k.txt'}
datasets = load_dataset('text', data_files=data_files, cache_dir='./data/')

Using custom data configuration default-235a9b97f9ebd10e
Reusing dataset text (./data/text/default-235a9b97f9ebd10e/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
# Unsupervised / Self-supervised dataset

column_names = datasets["train"].column_names
sent0_cname = column_names[0]
sent1_cname = column_names[0]

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

In [7]:
def prepare_features(examples):
    total = len(examples[sent0_cname])

    # Avoid "None" fields 
    for idx in range(total):
        if examples[sent0_cname][idx] is None:
            examples[sent0_cname][idx] = " "
        if examples[sent1_cname][idx] is None:
            examples[sent1_cname][idx] = " "

    sentences = examples[sent0_cname] + examples[sent1_cname]

    sent_features = tokenizer(
        sentences,
        max_length=max_len,
        truncation=True,
        padding=True,
    )

    features = {}
    for key in sent_features:
        features[key] = [[sent_features[key][i], sent_features[key][i+total]] for i in range(total)]

    return features

In [8]:
train_dataset = datasets["train"].map(prepare_features,
                                      batched=True,
                                    #   num_proc=24,
                                      remove_columns=column_names)

Loading cached processed dataset at ./data/text/default-235a9b97f9ebd10e/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-305eab4ce3c7d88c.arrow


In [9]:
train_dataset.num_rows

1000000

## 2. Contrastive Learning Model

In [10]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer, DistilBertModel, DistilBertPreTrainedModel, AutoConfig
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPooling

from distilface.modules.pooler import Pooler
from distilface.modules.similarity import Similarity


class DistilBertCLModel(DistilBertPreTrainedModel):
    def __init__(self, config, pooler_type=pooling_method, temp=temperature):
        super().__init__(config)

        self.config = config
        self.pooler_type = pooler_type
        self.temp = temperature

        self.distilbert = DistilBertModel(config)
        self.pooler = Pooler(pooler_type)
        self.sim = Similarity(temp=temp)

        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None):
        if self.training:
            return self.cl_forward(self.distilbert, input_ids, attention_mask)
        else:
            return self.sent_emb(self.distilbert, input_ids, attention_mask)

    def cl_forward(self, encoder, input_ids=None, attention_mask=None):
        batch_size = input_ids.size(0)
        num_sent = input_ids.size(1)  # Number of sentences in one instance: 2 sentences

        # Flatten all input tensors
        input_ids = input_ids.view((-1, input_ids.size(-1))) # (bs * num_sent, len)
        attention_mask = attention_mask.view((-1, attention_mask.size(-1))) # (bs * num_sent len)

        # Pre-trained Model Encoder
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )

        # Pooling
        pooler_output = self.pooler(attention_mask, outputs)
        pooler_output = pooler_output.view((batch_size, num_sent, pooler_output.size(-1)))  # (bs, num_sent, hidden)

        # Separate representation
        z1, z2 = pooler_output[:, 0], pooler_output[:, 1]

        # Cosine similarity
        cos_sim = self.sim(z1.unsqueeze(1), z2.unsqueeze(0))

        # Calculate contrastive loss
        criterion = nn.CrossEntropyLoss()
        labels = torch.arange(cos_sim.size(0)).long().to(self.device)
        loss = criterion(cos_sim, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=cos_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

    def sent_emb(self, encoder, input_ids=None, attention_mask=None):
        outputs = encoder(
            input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True,
            return_dict=True,
        )
        pooler_output = self.pooler(attention_mask, outputs)

        return BaseModelOutputWithPooling(
            pooler_output=pooler_output,
            last_hidden_state=outputs.last_hidden_state,
            hidden_states=outputs.hidden_states,
        )


pretrained_model_name = 'distilbert-base-uncased'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config = AutoConfig.from_pretrained(pretrained_model_name)

model = DistilBertCLModel.from_pretrained(pretrained_model_name, config=config).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

model.eval();

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertCLModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertCLModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertCLModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### 2.1 Initial DistilBERT embeddings performance

In [11]:
import senteval


def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [" ".join(s) for s in batch]
    batch = tokenizer.batch_encode_plus(
        sentences,
        return_tensors="pt",
        padding=True,
    )

    for k in batch:
        batch[k] = batch[k].to(device)

    with torch.no_grad():
        outputs = model(**batch)

    pooled_result = outputs.pooler_output.cpu()

    return pooled_result


def evaluate_model():
    PATH_TO_DATA = "./data"

    params = {"task_path": PATH_TO_DATA, "usepytorch": True, "kfold": 10}
    tasks = ["STSBenchmark", 'STS12', 'STS13', 'STS14', 'STS15']

    se = senteval.engine.SE(params, batcher, prepare)
    results = se.eval(tasks)

    print('STS12: ', results["STS12"]["all"]["spearman"]["all"])
    print('STS13: ', results["STS13"]["all"]["spearman"]["all"])
    print('STS14: ', results["STS14"]["all"]["spearman"]["all"])
    print('STS15: ', results["STS15"]["all"]["spearman"]["all"])
    print('STSB: ', results["STSBenchmark"]["test"]["spearman"][0])

    return results

In [12]:
results = evaluate_model()
results

/opt/conda/lib/python3.8/site-packages/SentEval-0.1.0-py3.8.egg/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/conda/lib/python3.8/site-packages/SentEval-0.1.0-py3.8.egg/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


STS12:  0.45494575136623244
STS13:  0.6251790603640242
STS14:  0.5451448337273794
STS15:  0.6774550565775094
STSB:  0.5881473476501239


{'STSBenchmark': {'train': {'pearson': (0.6256178513170415, 0.0),
   'spearman': SpearmanrResult(correlation=0.5978875329612952, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.6913792568291351, 1.0139072213755932e-213),
   'spearman': SpearmanrResult(correlation=0.7000053692021727, pvalue=2.716295010782929e-221),
   'nsamples': 1500},
  'test': {'pearson': (0.5814356378176905, 1.576009021173952e-125),
   'spearman': SpearmanrResult(correlation=0.5881473476501239, pvalue=4.220866398589903e-129),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.6314438962690317,
    'mean': 0.6328109153212891,
    'wmean': 0.62998906548632},
   'spearman': {'all': 0.6190266881445151,
    'mean': 0.6286800832711973,
    'wmean': 0.6140842226712176}}},
 'STS12': {'MSRpar': {'pearson': (0.3684255146280718, 1.586313914019332e-25),
   'spearman': SpearmanrResult(correlation=0.41797231160911885, pvalue=4.504689780390467e-33),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.6300413311517139, 3.2

## 3. Trainer

In [13]:
# import mlflow

from transformers import Trainer, TrainingArguments
from transformers import default_data_collator

training_args = TrainingArguments(
    output_dir='output',
    overwrite_output_dir=True,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    max_steps=max_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_strategy='steps',
    save_steps=max_steps,
    fp16=fp16,
)

model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator
)

max_steps is given, it will override any value given in num_train_epochs
Using amp half precision backend


In [14]:
train_result = trainer.train()

/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 30000


Step,Training Loss
500,0.086700
1000,0.012000
1500,0.006900
2000,0.004700
2500,0.003500
3000,0.002700
3500,0.002100
4000,0.001800
4500,0.001500
5000,0.001300


Saving model checkpoint to output/checkpoint-30000
Configuration saved in output/checkpoint-30000/config.json
Model weights saved in output/checkpoint-30000/pytorch_model.bin
tokenizer config file saved in output/checkpoint-30000/tokenizer_config.json
Special tokens file saved in output/checkpoint-30000/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




In [15]:
import pandas as pd

pd.DataFrame(trainer.state.log_history).to_csv(f'logs/cl-distilled_pooling_{pooling_method}_fp16.csv', index=False)

## 4. Evaluate DistilBert CL Model performance

In [16]:
model.eval()

results = evaluate_model()
results

/opt/conda/lib/python3.8/site-packages/SentEval-0.1.0-py3.8.egg/senteval/sts.py:42: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
/opt/conda/lib/python3.8/site-packages/SentEval-0.1.0-py3.8.egg/senteval/sts.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


STS12:  0.5813589344321862
STS13:  0.7364919075007814
STS14:  0.6329844785632407
STS15:  0.7255777346829861
STSB:  0.6351463091290117


{'STSBenchmark': {'train': {'pearson': (0.6951318988594317, 0.0),
   'spearman': SpearmanrResult(correlation=0.6705592635131801, pvalue=0.0),
   'nsamples': 5749},
  'dev': {'pearson': (0.7083963314426069, 6.3442014660101365e-229),
   'spearman': SpearmanrResult(correlation=0.7089183907372294, pvalue=2.0823864854419737e-229),
   'nsamples': 1500},
  'test': {'pearson': (0.6471429278122012, 1.8291119756546653e-164),
   'spearman': SpearmanrResult(correlation=0.6351463091290117, pvalue=1.2065433768537708e-156),
   'nsamples': 1379},
  'all': {'pearson': {'all': 0.6899135654219469,
    'mean': 0.6835570527047466,
    'wmean': 0.6897679509920965},
   'spearman': {'all': 0.6771782540182998,
    'mean': 0.6715413211264737,
    'wmean': 0.6715680983231368}}},
 'STS12': {'MSRpar': {'pearson': (0.482723202943209, 4.8767788847783317e-45),
   'spearman': SpearmanrResult(correlation=0.4785734920966143, pvalue=3.4232583274527026e-44),
   'nsamples': 750},
  'MSRvid': {'pearson': (0.7849726951570825